In [1]:
class signature_oracle:
    def __init__(self, g, q, N):
        self.g = g
        self.sk = randint(1, N)
    def Sign(self, m):
        return (m, self.g^self.sk)
    def Verify(self, m, sigma):
        return (sigma==self.Sign(m))

In [2]:
# has secret x
class Prover:
    def __init__(self, g, q, N):
        self.honest = True
        self.g = g
        self.q = q
        self.k = randint(1, q)
        self.x = -99999
        
    def Setup(self, S):
        r = self.g^self.k
        return (r, S.Sign(r))
    
    def Prove(self, S, r, y):
        c = randint(1,self.q)
        s = self.k+c*self.x
        return (c, s, S.Sign(s))

In [3]:
class Verifier:
    def __init__(self, g, q, N):
        self.g = g

    def Verify(self, S, y, r, c, s, sigma_r, sigma_s):
        v1 = S.Verify(r, sigma_r)
        v2 = S.Verify(s, sigma_s)
        v3 = (r==self.g^s * y^(-c))
        return (v1 and v2 and v3)

In [4]:
import time
class NISZKP_Protocol:
    def __init__(self, bits, seed):
        start = 0
        end = 0
        
        set_random_seed(seed)
        current_randstate().set_seed_gp()
        
        N = 2^bits
        q = random_prime(N)
        Zq = Integers(q)
        g = Zq(2)
        
        # knowledge
        x = randint(1, q)
        # such that
        y = g^x
        
        prover = Prover(g, q, N)
        verifier = Verifier(g, q, N)
        S = signature_oracle(g, q, N)
        
        if prover.honest:
            prover.x = x
        else:
            prover.x = randint(1, q)

        start = time.time()
        r, sigma_r = prover.Setup(S)
        end = time.time()
        self.setup_time = end-start
        
        start = time.time()
        c, s, sigma_s = prover.Prove(S, r, y)
        end = time.time()
        self.prove_time = end-start
        
        start = time.time()
        self.result = verifier.Verify(S, y, r, c, s, sigma_r, sigma_s)
        end = time.time()
        self.verify_time = end-start

In [6]:
bits = 512
ctr = 0
iteration = 20
total_setup_time = 0
total_prove_time = 0
total_verify_time = 0

while ctr < iteration:
    seed = time.time_ns()
    print("iteration : %d / %d" % (ctr+1, iteration), end="\r")

    nizkp = NISZKP_Protocol(bits, seed)
    if nizkp.result:
        ctr += 1
        total_setup_time += nizkp.setup_time*10^6
        total_prove_time += nizkp.prove_time*10^6
        total_verify_time += nizkp.verify_time*10^6
    else:
        pass

print("\n-----------------------------")
print("Total setup time / Average setup time         : %.2fus / %.2fus" % (total_setup_time, total_setup_time/iteration))
print("Total prove time / Average prove time         : %.2fus / %.2fus" % (total_prove_time, total_prove_time/iteration))
print("Total verify time / Average verify time       : %.2fus / %.2fus" % (total_verify_time, total_verify_time/iteration))


iteration : 20 / 20
-----------------------------
Total setup time / Average setup time         : 3505.95us / 175.30us
Total prove time / Average prove time         : 2056.60us / 102.83us
Total verify time / Average verify time       : 8985.04us / 449.25us
